In [1]:
import os
import time
import pandas as pd
import numpy as np

In [2]:
import sklearn
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler

from trustee import RegressionTrustee
import matplotlib.pyplot as plt

In [3]:
# Load data
df = pd.read_csv("./model_trainer.csv")
df.head(10)

,file,Number of Packets,Median Time Between Packets,Bytes Down,Bytes Up,Number of Flows,Average Download Speed
0,0,4193.0,0.000101,0.0,0.0,65.0,643.390
1,1,4228.0,0.000096,15752901.0,227241.0,63.0,763.560
2,10,15917.0,0.000028,17191704.0,355968.0,65.0,65.950
3,100,3955.0,0.000095,15232625.0,220646.0,63.0,709.970
4,101,4664.0,0.000116,15754546.0,252117.0,61.0,830.560
5,102,4731.0,0.000110,15767568.0,253321.0,65.0,708.265
6,103,3999.0,0.000108,15732220.0,228815.0,65.0,437.610
7,104,4665.0,0.000125,15768401.0,248710.0,67.0,792.555
8,105,4786.0,0.000115,15770027.0,249718.0,63.0,782.310
9,106,4560.0,0.000122,15749099.0,253585.0,63.0,604.795


In [4]:
# Scaling features
scaler = StandardScaler()
target_variable = 'Average Download Speed'
train_features = list(set(df.columns) - {target_variable} - {'file'} - {'Number of Flows'})

In [5]:
# Split data into training and testing sets
X = df[train_features]
y = df[target_variable]
X_scaled = scaler.fit_transform(X)  # Scale the features
x_train, x_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [6]:
# Train the model
rgr = MLPRegressor(random_state=42, max_iter=1000)  # You can tune other parameters as well
rgr.fit(x_train, y_train)

/home/ziv/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(max_iter=1000, random_state=42)

In [7]:
# Validate the model
y_pred = rgr.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 24054.621916933425


In [8]:
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 155.0955251351032


In [9]:
mean_y_test = y_test.mean()
print("Mean of Test Data:", mean_y_test)


Mean of Test Data: 406.6859322033898


In [10]:
# Assuming rmse and mean_y_test have been calculated
rmsep = (rmse / mean_y_test) * 100
print("Root Mean Squared Error Percentage:", rmsep, "%")


Root Mean Squared Error Percentage: 38.136437199784325 %


In [15]:
x_train[:10]

array([[ 0.29679036,  0.14998838, -0.8880802 , -0.71660312],
       [ 0.26253174,  0.16997308, -0.70581755, -0.24843312],
       [ 0.94770426,  0.15938789, -0.82289137, -0.65992731],
       [ 0.35160417,  0.170834  , -0.8299234 , -0.75538283],
       [ 0.27623519, -0.79213884, -0.91183707, -0.71436973],
       [10.38938159, -1.13049783,  1.30173242,  3.08879728],
       [ 0.82437321,  0.1648792 , -0.80160522, -0.68076742],
       [-0.02524072,  0.55737724, -0.81852011, -0.71969215],
       [-0.77893049,  0.47770578,  1.36445054,  1.28913741],
       [-0.77893049,  0.48031955,  1.33537214,  1.22099012]])

In [12]:

# Create and Train a Trustee Tree
trustee = RegressionTrustee(expert=rgr)
trustee.fit(x_train, y_train, num_samples=len(x_train) // 2, num_iter=20, train_size=0.99)

# Display Trustee Results
_, dt, _, score = trustee.explain()
print(f"Training score of pruned DT: {score}")
dt_y_pred = dt.predict(x_train)

# plot a tree
fig = plt.figure(figsize=(250,200))
plot_tree(dt, feature_names=X.columns, class_names=df[target_variable], filled=True, max_depth=7)

Training score of pruned DT: 1.0


[Text(0.49, 0.9444444444444444, 'Median Time Between Packets <= -0.149\nsquared_error = 96485.951\nsamples = 114\nvalue = 453.59'),
 Text(0.24, 0.8333333333333334, 'Bytes Down <= 0.032\nsquared_error = 247.462\nsamples = 43\nvalue = 59.794'),
 Text(0.16, 0.7222222222222222, 'Median Time Between Packets <= -0.429\nsquared_error = 18.225\nsamples = 12\nvalue = 35.629'),
 Text(0.12, 0.6111111111111112, 'Median Time Between Packets <= -0.71\nsquared_error = 1.44\nsamples = 11\nvalue = 36.868'),
 Text(0.08, 0.5, 'Bytes Down <= -0.441\nsquared_error = 0.457\nsamples = 10\nvalue = 37.188'),
 Text(0.04, 0.3888888888888889, 'squared_error = 0.053\nsamples = 3\nvalue = 36.245'),
 Text(0.12, 0.3888888888888889, 'squared_error = 0.085\nsamples = 7\nvalue = 37.593'),
 Text(0.16, 0.5, 'squared_error = -0.0\nsamples = 1\nvalue = 33.668'),
 Text(0.2, 0.6111111111111112, 'squared_error = 0.0\nsamples = 1\nvalue = 21.992'),
 Text(0.32, 0.7222222222222222, 'Bytes Down <= 0.689\nsquared_error = 22.641\nsa